In [1]:
#!/usr/bin/env python
# coding: utf-8

# --- NOTES -------------------------------------------------------------------
# 1. Update the datasets, dataList
# -----------------------------------------------------------------------------

import os
import re
import sys
import json
import time
import pyspark
from ast import literal_eval
from copy import deepcopy
from datetime import datetime
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.functions import udf, unix_timestamp, col ,length
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, FloatType, DateType, TimestampType
from pyspark.sql.functions import mean as _mean, stddev as _stddev, col
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline 
from collections import Counter
#import spacy
#from spacy import displacy
#import en_core_web_sm


In [2]:
# -----------------------------------------------------------------------------
# --- Function Definitions Begin ----------------------------------------------

# Function to find mean and stdv for all files
def mean_stdv(df):
    unlist = udf(lambda x: round(float(list(x)[0]),3), DoubleType())
    for i in ["count"]:
        assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")
        scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")
        pipeline = Pipeline(stages=[assembler, scaler])
        df = pipeline.fit(df).transform(df).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")
        df_stats = df.select(_mean(col('count_Scaled')).alias('mean'),_stddev(col('count_Scaled')).alias('std')).collect()
        mean = df_stats[0]['mean']
        std = df_stats[0]['std']
        return df_stats 

# Function to sum all count of values for all files
def count_all_values(df):
    res = df.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
    return res

# Regex function to check website type
def re_find_website(df,count_all,found_type):
    web_re_rexpr = "WWW\.|\.COM|HTTP\:"
    df_filtered = df.filter(df["val"].rlike(web_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["website"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function to check zip type
def re_find_zipCode(df,count_all,found_type):
    zip_re_rexpr = "^\d{5}?$|^\d{5}?-\d\d\d$|^\d{8}?$"
    df_filtered = df.filter(df["val"].rlike(zip_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["zip_code"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function to check buildingCode type
def re_find_buildingCode(df,count_all,found_type):
    bc_re_rexpr = "([A-Z])\d\-"
    df_filtered = df.filter(df["val"].rlike(bc_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["building_classification"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0 

# Regex function to check phone number type
def re_find_phoneNum(df,count_all,found_type):
    phone_re_rexpr = "^\d{10}?$|^\(\d\d\d\)\d\d\d\d\d\d\d$|^\d\d\d\-\d\d\d\-\d\d\d\d$"
    df_filtered = df.filter(df["val"].rlike(phone_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["phone_number"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function to check lat_lon type
def re_find_lat_lon(df,count_all,found_type):
    ll_re_rexpr = "\([-+]?[0-9]+\.[0-9]+\,\s*[-+]?[0-9]+\.[0-9]+\)"
    df_filtered = df.filter(df["val"].rlike(ll_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["lat_lon_cord"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function to check street_addrees type
def re_find_street_address(df,count_all,col_length,found_type):
    st_re_rexpr = "\sROAD|\sSTREET|\sPLACE|\sDRIVE|\sBLVD|\sST|\sRD|\sDR|\sAVENUE|\sAVE"
    df_filtered = df.filter(df["val"].rlike(st_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.8): 
            if (col_length >= 15):
                found_type = found_type + ["address"]
            elif (col_length < 15):
                found_type = found_type + ["street"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function to check school name type
def re_find_school(df,count_all,found_type):
    school_re_rexpr = "\sSCHOOL|\sACADEMY|HS\s|ACAD|I.S.\s|IS\s|M.S.\s|P.S\s|PS\s|ACADEMY\s"
    df_filtered = df.filter(df["val"].rlike(school_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.5): 
            found_type = found_type + ["school_name"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function for checking house number 
def re_find_houseNo(df,count_all,found_type):
    houseNo_re_rexpr = "^\d{2}?$|^\d{3}?$|^\d{4}?$"
    df_filtered = df.filter(df["val"].rlike(houseNo_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["house number"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function for checking school subject
def re_find_school_subject(df,count_all,found_type):
    school_subj_re_rexpr = "^ENGLISH$|^ENGLISH\s[0-9]?$|^MATH\s[A-Z$]|^MATH$|^SCIENCE$|^SOCIAL\sSTUDIES$|^ALGEBRA\s[A-Z]$|\
                            ^CHEMISTRY$|^ASSUMED\sTEAM\sTEACHING$|^EARTH\sSCIENCE$|^GEOMETRY$|^ECONOMICS$|^GLOBAL HISTORY$|\
                            ^GLOBAL\sHISTORY[A-Z]$|^LIVING ENVIRONMENT$|^PHYSICS$|^US\sGOVERNMENT$|^US\sGOVERNMENT$|^US\sGOVERNMENT\s&|\
                            ^US\SHISTORY$|^GLOBAL HISTORY\s[0-9]?$"
    df_filtered = df.filter(df["val"].rlike(school_subj_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.5): 
            found_type = found_type + ["school subject"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

# Regex function for checking school level 
def re_find_schoolLevel(df,count_all,found_type):
    schlvl_re_rexpr = "^[K]\-\d?$|^HIGH SCHOOL$|^ELEMENTARY$|^ELEMENTARY SCHOOL$|^MIDDLE SCHOOL$|^TRANSFER\sSCHOOL$|^MIDDLE$|^HIGH\sSCHOOL\sTRANSFERL$|^YABC$|^[K]\-[0-9]{2}$"
    df_filtered = df.filter(df["val"].rlike(schlvl_re_rexpr))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        if (res >= 0.85): 
            found_type = found_type + ["school level"]
        return res, found_type, count_filtered
    else:
        return 0, found_type, 0

# --- Functions FOR NLP Starts HERE -------------------------------------------
def nlp_find_person(df,count_all,found_type):
    #Your Code HERE: 
    #Use count_all for percentage calculation
    #Please return two values: (1)percentage of such type in this col AND (2)the type found for this column
    #if found:
#         found_type = found_type + ["person"]
    #if not found:
    return 0, found_type, 0

def nlp_find_business_name(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_vehicle_type(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_color(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_car_make(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_car_model(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_neighborhood(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_borough(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

def nlp_find_city(df,count_all,found_type):
    #Your Code HERE:
    return 0, found_type, 0

# --- Function FOR NLP End ------------------------------------------------

# --- Functions FOR LIST COMPARISON Starts HERE -------------------------------
def list_find_school_subject(df,count_all,found_type):
    df_filtered = df.filter(df["val"].isin(ss_keywords))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.4): 
            found_type = found_type + ["school subject"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

def list_find_business_name(df,count_all,found_type):
    #Your Code HERE: 
    return 0, found_type, 0

def list_find_neighborhood(df,count_all,found_type):
    df_filtered = df.filter(df["val"].isin(nh_keywords))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.1): 
            found_type = found_type + ["neighborhood"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

def list_find_area_of_study(df,count_all,found_type):
    df_filtered = df.filter(df["val"].isin(aos_keywords))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.3): 
            found_type = found_type + ["area of study"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0
    

def list_find_agency(df,count_all,found_type):
    df_filtered = df.filter(df["val"].isin(ca_keywords))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.1): 
            found_type = found_type + ["city agency"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

def list_find_location_type(df,count_all,found_type):
    df_filtered = df.filter(df["val"].isin(lt_keywords))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.1): 
            found_type = found_type + ["location type"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

def list_find_parks_playgrounds(df,count_all,found_type):
    df_filtered = df.filter(df["val"].isin(pp_keywords))
    if (df_filtered.count() is not 0):
        count_filtered = df_filtered.rdd.map(lambda x: (1,x[1])).reduceByKey(lambda x,y: x + y).collect()[0][1]
        res = float(count_filtered/count_all)
        print(res)
        if (res >= 0.1): 
            found_type = found_type + ["parks and playgrounds"]
        return res, found_type, count_filtered 
    else:
        return 0, found_type, 0

def import_keyword_list(inputDir):
    klist = sc.textFile(inputDir)
    klist = klist.flatMap(lambda x: x.split(",")).collect()
    klist = [x.strip('"') for x in klist]
    klist = [re.sub("\[|\]|\'|\'|" "", "", item)for item in klist]
    klist = [re.sub(" " "", "", item)for item in klist]
    return(klist)


# --- Function Definitions End ------------------------------------------------
# -----------------------------------------------------------------------------



In [3]:
# -----------------------------------------------------------------------------
# --- MAIN --------------------------------------------------------------------

if __name__ == "__main__":
    # Setting spark context and 
    sc = SparkContext()
    spark = SparkSession \
        .builder \
        .appName("project_task2") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    sqlContext = SQLContext(sparkContext=spark.sparkContext, sparkSession=spark)


In [4]:



    # Current user path
    env_var = os.environ
    this_user = env_var['USER']

    # Input & output directories
    #inputDirectory = "/user/hm74/NYCColumns/"#sys.argv[1]
    #outputDirectory = "/user/" + this_user + "/project/task2/"#sys.argv[2]
    inputDirectory = "/home/ted/school/big_data/project/big_data_course_project/task2/raw_data/"
    inputFileClusters = "/home/ted/school/big_data/project/big_data_course_project/task2/resources/filename_clusters.json"
    input_pp_keywords = "park_playground_keywords"
    input_aos_keywords = "area_of_study_keywords"
    input_ca_keywords = "city_agency_keywords"
    input_ss_keywords = "school_subject_keywords"
    input_sn_keywords = "school_name_keywords"
    input_lt_keywords = "location_type_keywords"
    input_nh_keywords = "neighborhood_keywords"
    
    pp_keywords = import_keyword_list(input_pp_keywords) # parks & playgrounds
    aos_keywords = import_keyword_list(input_aos_keywords) # area of study
    ca_keywords = import_keyword_list(input_ca_keywords) # city agency
    ss_keywords = import_keyword_list(input_ss_keywords) # school subject
    sn_keywords = import_keyword_list(input_sn_keywords) # school name
    lt_keywords = import_keyword_list(input_lt_keywords) # location type
    nh_keywords = import_keyword_list(input_nh_keywords) # neighborhood
    
    
    # Output JSON Semantic Schema
    jsonSchema = {
        "column_name": "",
        "semantic_type": [],
        "count": 0
    }

    # Inner semantic schema 
    semanticSchema = {
        "semantic_type": "",
        "label": "",
        "count": 0 
    }

    # Importing cluster3 format it and put it into a list
    #raw_data = sc.textFile("/user/aj2885/Project_Resource/cluster3_labels.tsv")
    raw_data = sc.textFile("true_labels.tsv")
    raw_list = raw_data.map(lambda x: x.split("\t")).collect()

    # Iteration over dataframes begins bu using dataframe file names
    processCount = 1

    # Create schema for raw data before reading into df 
    customSchema = StructType([
                StructField("val", StringType(), True),
                StructField("count", IntegerType(), True)])

    

In [5]:
    #Testing first 10 files
    for filerow in raw_list:
        filename = filerow[0]
        labels = literal_eval(filerow[1])
        print("Processing Dataset =========== : ", str(processCount) + ' - ' +filename)
        # Read file to dataset and apply all regex functions
        found_type = []
        fileinfo = []
        regex_res = []
        df = sqlContext.read.format("csv").option("header","false").option("inferSchema", "true").option("delimiter", "\t").schema(customSchema).load(inputDirectory + filename)
        df_stats = mean_stdv(df)
        mean = df_stats[0]['mean']
        std = df_stats[0]['std']
        count_all = count_all_values(df)

        #added col_length which is the average length of the col
        df_length = df.select(_mean(length(col("val"))).alias('avg_length'))
        col_length= df_length.collect()[0][0]

        percentage_website, found_type, type_count_web = re_find_website(df,count_all,found_type)
        percentage_zip, found_type, type_count_zip = re_find_zipCode(df,count_all,found_type)
        percentage_buildingCode, found_type,type_count_building = re_find_buildingCode(df,count_all,found_type)
        percentage_phoneNum, found_type, type_count_phone = re_find_phoneNum(df,count_all,found_type)
        percentage_lat_lon, found_type, type_count_lat_lon = re_find_lat_lon(df,count_all,found_type)
        percentage_add_st, found_type, type_count_add_st = re_find_street_address(df,count_all,col_length,found_type)
        percentage_school_name, found_type, type_count_school_name= re_find_school(df,count_all,found_type)
        percentage_house_no, found_type ,type_count_house_no= re_find_houseNo(df,count_all,found_type)
        percentage_school_lvl, found_type, type_count_school_lvl= re_find_schoolLevel(df,count_all,found_type)
        percentage_school_subject, found_type, type_count_school_subject= re_find_school_subject(df,count_all,found_type)
        
        # moved this block up here -ted
        percentage_area_of_study, found_type, type_count_area_of_study = list_find_area_of_study(df,count_all,found_type)
        percentage_school_subject, found_type, type_count_school_subject= list_find_school_subject(df,count_all,found_type)
        percentage_agency, found_type, type_count_agency= list_find_agency(df,count_all,found_type)
        percentage_location, found_type, type_count_location= list_find_location_type(df,count_all,found_type)
        percentage_neighborhood, found_type, type_count_neighborhood= list_find_neighborhood(df,count_all,found_type)
        percentage_parks_playgrounds, found_type, type_count_parks_playgrounds = list_find_parks_playgrounds(df,count_all,found_type)
        
        type_count = type_count_web + type_count_zip + type_count_building + type_count_phone + \
                    type_count_lat_lon + type_count_add_st + type_count_school_name + \
                    type_count_house_no + type_count_school_lvl + type_count_school_subject + \
                    type_count_area_of_study + type_count_neighborhood + type_count_agency + \
                    type_count_location + type_count_parks_playgrounds
        
        #give a default value for all other precentages 
        percentage_person = 0
        percentage_business_name = 0
        percentage_vehicle_type = 0
        percentage_color = 0
        percentage_car_make = 0
        percentage_car_model = 0
        #percentage_neighborhood = 0
        percentage_borough= 0 
        percentage_city = 0
        #percentage_area_of_study = 0
        #percentage_location = 0
        #percentage_agency = 0
        #percentage_parks_playgrounds = 0

        #STEP TWO: NLP LABEL AND LIST CHECK
        # if not found_type:
        #     #ANKUSH PART: NLP CHECK TYPES
        #     percentage_person, found_type, type_count_person = nlp_find_person(df,count_all,found_type)
        #     percentage_business_name, found_type, type_count_business = nlp_find_business_name(df,count_all,found_type)
        #     percentage_vehicle_type, found_type, type_count_vehicle_type = nlp_find_vehicle_type(df,count_all,found_type)
        #     percentage_color, found_type, type_count_color = nlp_find_color(df,count_all,found_type)
        #     percentage_car_make, found_type, type_count_car_make = nlp_find_car_make(df,count_all,found_type)
        #     percentage_car_model, found_type, type_count_car_model = nlp_find_car_model(df,count_all,found_type)
        #     percentage_neighborhood, found_type, type_count_neighborhood = nlp_find_neighborhood(df,count_all,found_type)
        #     percentage_borough, found_type, type_count_borough = nlp_find_borough(df,count_all,found_type)
        #     percentage_city, found_type, type_count_city = nlp_find_city(df,count_all,found_type)
        
        #     #TED PART: LIST or SIMILARITY CHECK TYPEs
        #   percentage_school_subject, found_type, type_count_school_subject= list_find_school_subject(df,count_all,found_type)
        #     percentage_business_name, found_type, type_count_business= list_find_business_name(df,count_all,found_type)
        #   percentage_neighborhood, found_type, type_count_neighborhood= list_find_neighborhood(df,count_all,found_type)
        #   percentage_area_of_study, found_type, type_count_area_of_study = list_find_area_of_study(df,count_all,found_type)
        #   percentage_agency, found_type, type_count_agency= list_find_agency(df,count_all,found_type)
        #   percentage_location, found_type, type_count_location= list_find_location_type(df,count_all,found_type)
        #   percentage_parks_playgrounds, type_count_location_parks_playgrounds = list_find_parks_playgrounds(df,count_all,found_type
        # !!! NOTE: Please remeber to add type_count_XXX back to type_count in LINE 347
        fileinfo.extend([filename,mean,std,count_all,col_length, percentage_website, percentage_zip,percentage_buildingCode,percentage_phoneNum,percentage_lat_lon,percentage_add_st,percentage_school_name,percentage_house_no,percentage_school_lvl,percentage_person,percentage_school_subject,percentage_vehicle_type, percentage_color,percentage_car_make,percentage_car_model,percentage_neighborhood,percentage_borough,percentage_city,percentage_business_name,percentage_area_of_study,percentage_location,percentage_parks_playgrounds,found_type, type_count])
        regex_res.append(fileinfo)
        print(regex_res)
        # USE ME to export the JSON for current dataset
        print("Saving Dataset =============== : ", str(processCount) + ' - ' +filename)
        processCount += 1
        #outJSON = deepcopy(jsonSchema)
        #outJSON["column_name"] = filename
        #outJSON["semantic_type"] = found_type
        #outJSON["count"] = type_count
        #outJSON = sc.parallelize([json.dumps(outJSON)])
        #outJSON.saveAsTextFile(outputDirectory + filename + '/task2.json')



    # Output regex function result 
    rdd = sc.parallelize(regex_res)
    row_rdd = rdd.map(lambda x: Row(x))
    df = row_rdd.toDF()
    df = df.select(col('_1').alias('coln'))
    length = len(df.select('coln').take(1)[0][0])
    df = df.select([df.coln[i] for i in range(length)])
    df = df.select(col('coln[0]').alias('filename'),col('coln[1]').alias('mean'),col('coln[2]').alias('stdv'),
               col('coln[3]').alias('count_all'),col('coln[4]').alias('col_length'),col('coln[5]').alias('precentage_website'),
               col('coln[6]').alias('precentage_zip'),col('coln[7]').alias('percentage_buildingCode'),col('coln[8]').alias('percentage_phoneNum'),
               col('coln[9]').alias('percentage_lat_lon'),col('coln[10]').alias('percentage_add_st'),col('coln[11]').alias('percentage_school_name'),
               col('coln[12]').alias('percentage_houseNo'),col('coln[13]').alias('percentage_school_lvl'),col('coln[14]').alias('percentage_person'),
               col('coln[15]').alias('percentage_school_subject'),col('coln[16]').alias('percentage_vehicle_type'),col('coln[17]').alias('percentage_color'),
               col('coln[18]').alias('percentage_car_make'),col('coln[19]').alias('percentage_car_model'),
               col('coln[20]').alias('percentage_neighborhood'),col('coln[21]').alias('percentage_borough'),col('coln[22]').alias('percentage_city'),
               col('coln[23]').alias('percentage_business_name'),col('coln[24]').alias('percentage_area_of_study'),col('coln[25]').alias('percentage_location_type'),
               col('coln[26]').alias('percentage_parks_playgrounds'),col('coln[27]').alias('types'), col('coln[28]').alias('types_count')
               )

    types_found_count = df.where(col('types') > " ").count()
    print(types_found_count)
    #df.write.csv('regex_res.csv')


Processing Dataset =========== :  1 - vw9i-7mzq.interest3.txt.gz
0.2676056338028169
0.04225352112676056
[['vw9i-7mzq.interest3.txt.gz', 0.29444444444444445, 0.3207935422230724, 71, 16.88888888888889, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.2676056338028169, 0.04225352112676056, 0, [], 22]]
Saving Dataset =============== :  1 - vw9i-7mzq.interest3.txt.gz
Processing Dataset =========== :  2 - tyfh-9h2y.BROOKLYN___COOPERATIVES_COMPARABLE_PROPERTIES___Building_Classification.txt.gz
[['tyfh-9h2y.BROOKLYN___COOPERATIVES_COMPARABLE_PROPERTIES___Building_Classification.txt.gz', 0.3785, 0.47786016085601163, 958, 10.5, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['building_classification'], 958]]
Saving Dataset =============== :  2 - tyfh-9h2y.BROOKLYN___COOPERATIVES_COMPARABLE_PROPERTIES___Building_Classification.txt.gz
Processing Dataset =========== :  3 - w7w3-xahh.Location.txt.gz
[['w7w3-xahh.Location.txt.gz', 8.022479922576566e-05, 0.00413211825588

0.28636006289308175
0.34394654088050314
0.2456761006289308
[['p2d7-vcsb.ACCOUNT_CITY.txt.gz', 0.01819801980198018, 0.07556178382538213, 5088, 9.58085808580858, 0, 0, 0, 0, 0, 0.005699685534591195, 0.00039308176100628933, 0, 0, 0, 0, 0, 0, 0, 0, 0.34394654088050314, 0, 0, 0, 0, 0, 0.2456761006289308, ['city agency', 'neighborhood', 'parks and playgrounds'], 4488]]
Saving Dataset =============== :  19 - p2d7-vcsb.ACCOUNT_CITY.txt.gz
Processing Dataset =========== :  20 - qcdj-rwhu.BUSINESS_NAME2.txt.gz
[['qcdj-rwhu.BUSINESS_NAME2.txt.gz', 0.008068647540983607, 0.052338488074875206, 1039, 13.487704918032787, 0, 0, 0, 0, 0, 0.03176130895091434, 0.007699711260827719, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, [], 41]]
Saving Dataset =============== :  20 - qcdj-rwhu.BUSINESS_NAME2.txt.gz
Processing Dataset =========== :  21 - kj4p-ruqc.StreetName.txt.gz
3.050463823024291e-06
3.660556587629149e-05
3.050463823024291e-06
[['kj4p-ruqc.StreetName.txt.gz', 0.025680065524193614, 0.06083381422813

[['wg9x-4ke6.Principal_phone_number.txt.gz', 0.001086760280842527, 0.02255314279897806, 2190, 11.995987963891675, 0, 0, 0, 0.9584474885844749, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['phone_number'], 2099]]
Saving Dataset =============== :  38 - wg9x-4ke6.Principal_phone_number.txt.gz
Processing Dataset =========== :  39 - cyfw-hfqk.STATEN_ISLAND_CONDOMINIUM_PROPERTY_Neighborhood.txt.gz
0.25
[['cyfw-hfqk.STATEN_ISLAND_CONDOMINIUM_PROPERTY_Neighborhood.txt.gz', 0.15155555555555555, 0.32460826819071903, 24, 12.11111111111111, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0, 0, 0, ['neighborhood'], 6]]
Saving Dataset =============== :  39 - cyfw-hfqk.STATEN_ISLAND_CONDOMINIUM_PROPERTY_Neighborhood.txt.gz
Processing Dataset =========== :  40 - sqcr-6mww.Cross_Street_1.txt.gz
7.506404571614853e-06
1.0723435102306932e-06
[['sqcr-6mww.Cross_Street_1.txt.gz', 0.006666392769104305, 0.029173850509321152, 932537, 12.995891536565324, 0, 0, 0, 0, 0, 0.8668224424339195, 

0.18827611395178961
0.06525444363282201
[['i8ys-e4pm.CORE_COURSE_9_12_ONLY_.txt.gz', 0.08264999999999999, 0.21670821396523024, 16428, 10.45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.06525444363282201, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, [], 1072]]
Saving Dataset =============== :  58 - i8ys-e4pm.CORE_COURSE_9_12_ONLY_.txt.gz
Processing Dataset =========== :  59 - jt7v-77mi.Vehicle_Make.txt.gz
1.1065493001462969e-07
7.745845101024079e-07
6.639295800877781e-07
0.0001751667542131588
3.9614464945237425e-05
4.4704591725910394e-05
[['jt7v-77mi.Vehicle_Make.txt.gz', 0.0010480139676996949, 0.02121762121099167, 9037103, 4.34730103302779, 0, 8.852394401170375e-07, 1.1065493001462969e-07, 0, 0, 6.639295800877781e-07, 2.766373250365742e-06, 5.090126780672966e-06, 0, 0, 7.745845101024079e-07, 0, 0, 0, 0, 3.9614464945237425e-05, 0, 0, 0, 1.1065493001462969e-07, 0.0001751667542131588, 4.4704591725910394e-05, [], 2445]]
Saving Dataset =============== :  59 - jt7v-77mi.Vehicle_Make.txt.gz
Processing Dataset ======

3.4516519347285714e-07
2.1572824592053573e-06
0.0001787092789205718
1.8984085641007144e-06
[['a5td-mswe.Vehicle_Color.txt.gz', 0.002353610503282274, 0.03732129798318185, 11588654, 3.833698030634573, 0, 0, 0, 0, 0, 3.4516519347285714e-07, 0, 1.7258259673642857e-07, 0, 0, 0, 0, 0, 0, 0, 0.0001787092789205718, 0, 0, 0, 0, 2.1572824592053573e-06, 1.8984085641007144e-06, [], 2128]]
Saving Dataset =============== :  76 - a5td-mswe.Vehicle_Color.txt.gz
Processing Dataset =========== :  77 - jz4z-kudi.Respondent_Address__City_.txt.gz
0.4852950761856464
1.0297096990930832e-06
0.5898158621630597
0.5135474271416031
[['jz4z-kudi.Respondent_Address__City_.txt.gz', 0.00020390395386236295, 0.010523727904105435, 9711475, 9.755947429712194, 0, 1.5445645486396248e-06, 0, 2.0594193981861663e-07, 0, 0.0007653832193358888, 3.871708468589993e-05, 8.237677592744665e-07, 2.0594193981861663e-07, 0, 0, 0, 0, 0, 0, 0.5898158621630597, 0, 0, 0, 0, 1.0297096990930832e-06, 0.5135474271416031, ['city agency', 'neigh

[['ci93-uc8s.Website.txt.gz', 0.0005503558988338636, 0.014130831571006733, 6654, 27.840678479479024, 0.9954914337240758, 0, 0.00030057108506161706, 0, 0, 0.0004508566275924256, 0.0021039975954313195, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['website'], 6643]]
Saving Dataset =============== :  95 - ci93-uc8s.Website.txt.gz
Processing Dataset =========== :  96 - n2s5-fumm.BRONX_CONDOMINIUM_PROPERTY_Building_Classification.txt.gz
[['n2s5-fumm.BRONX_CONDOMINIUM_PROPERTY_Building_Classification.txt.gz', 0.52, 0.5011985634456667, 54, 10.333333333333334, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['building_classification'], 54]]
Saving Dataset =============== :  96 - n2s5-fumm.BRONX_CONDOMINIUM_PROPERTY_Building_Classification.txt.gz
Processing Dataset =========== :  97 - cgz5-877h.SCHOOL_LEVEL_.txt.gz
[['cgz5-877h.SCHOOL_LEVEL_.txt.gz', 0.4779999999999999, 0.3948411072824105, 1543, 9.0, 0, 0, 0, 0, 0, 0, 0.2825664290343487, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

0.08841201716738198
[['735p-zed8.CANDMI.txt.gz', 0.2984166666666666, 0.3098738756987747, 116500, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.08841201716738198, 0, 0, 0, 0, 0, 0, [], 10300]]
Saving Dataset =============== :  115 - 735p-zed8.CANDMI.txt.gz
Processing Dataset =========== :  116 - t8hj-ruu2.Business_Phone_Number.txt.gz
[['t8hj-ruu2.Business_Phone_Number.txt.gz', 0.002490516866946839, 0.015247083634339442, 65101, 9.997997200907292, 0, 1.5360747146741216e-05, 0, 0.9985560897682063, 0, 0, 0, 0.00010752523002718852, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ['phone_number'], 65015]]
Saving Dataset =============== :  116 - t8hj-ruu2.Business_Phone_Number.txt.gz
Processing Dataset =========== :  117 - i5ef-jxv3.Agency.txt.gz
0.010704225352112675
0.7785915492957747
[['i5ef-jxv3.Agency.txt.gz', 0.3435625, 0.24034778933958495, 1775, 3.7291666666666665, 0, 0, 0, 0, 0, 0, 0, 0.007323943661971831, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.010704225352112675, 0, 0, ['city agency'], 141

Py4JJavaError: An error occurred while calling o86270.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5746.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5746.0 (TID 5754, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<count_double_VectorAssembler_0d7c2cc76290:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.s